# GIS Reconciliation for Wikidata
This notebook does the following:

1. Downloads a dataset from an ArcGIS FeatureServer REST API.
2. Converts all of the polygon data to a single point that is useful for Wikidata.
3. Attempts to find matching OpenStreetMap entities in the same place as the GIS data.
4. Writes it all out as a CSV that can be imported to OpenRefine.

In [ ]:
import logging
import sys

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [ ]:
import requests

GIS_SERVER_BASE_URL = 'https://gis.ccc.govt.nz/server/rest/services/OpenData/GreenAsset/FeatureServer/12/query'

logging.info("Retrieving data from GIS API")
raw_api_response = requests.get(
    GIS_SERVER_BASE_URL,
    params={
        'where': '1=1',                                  # cheeky trick to just get everything the server has
        'geometryType': 'esriGeometryEnvelope',
        'spatialRel': 'esriSpatialRelIntersects',
        'units': 'esriSRUnit_Metre',
        'outFields': '*',                                # get everything the server has
        'returnGeometry': 'true',                        # make sure we get the polygons
        'returnCentroid': 'true',                        # not very useful but may as well
        'f': 'geoJson',                                  # we need this format
    }
)

raw_api_response.raise_for_status()

raw_geojson = raw_api_response.json()
logging.info(f"Retrieved {len(raw_geojson['features'])} objects from GIS API")

In [ ]:
"""
Compute polylabels for all retrieved objects

If an object is a polygon, we want to coerce that to a single point.
This is because the Wikidata coordinate location (P625) property
expects a single point. 

To do this, we compute the "point of isolation" (aka a polylabel) for
the geometry. If the geometry is complex with multiple polygons we 
compute the polylabel for the largest polygon.

The polylabel represents (in most cases) the natural "visual centre" of
an object and therefore is a good choice for using in Wikidata.

NB: this process does not account for polygons with cutouts
"""
from functools import cmp_to_key
from typing import List, Tuple

from polylabel import polylabel
from shapely.geometry import Point, LinearRing, Polygon

COORD_PRECISION = 5  # more than 5 digits of precision is excessive for Wikidata
POLYLABEL_PRECISION = 0.00001  # this is plenty enough to reliably find a PoI


def convert_polygon(coordinates: List[Tuple[float, float]]) -> LinearRing:
    # remember! latitude = y, longitude = x, so we specify things as lon,lat
    points = [Point(lon, lat) for (lon, lat) in coordinates]
    return LinearRing(points)

entity_rows = []
logging.info(f'Computing coordinate location for all entries')
for park_response in raw_geojson["features"]:
    park_features = park_response["properties"]
    
    if park_response["geometry"]["type"] == "MultiPolygon":
        # complex geometry, with multiple polygons
        
        # first, compute polylabels for all polygons
        polygons = [convert_polygon(p[0]) for p in park_response["geometry"]["coordinates"]]
        
        # get the polygons ordered by size
        polygons = sorted(polygons, reverse=True, key=cmp_to_key(lambda p1, p2: Polygon(p1).area - Polygon(p2).area))
        
        coordinate_location = polylabel([polygons[0].coords], precision=POLYLABEL_PRECISION)
    elif park_response["geometry"]["type"] == "Polygon":
        coordinate_location = polylabel(park_response["geometry"]["coordinates"], precision=POLYLABEL_PRECISION)
    elif park_response["geometry"]["type"] == "Point":
        coordinate_location = park_response["geometry"]["coordinates"]
    else:
        continue
    park_features['coordinate_location'] = f'{round(coordinate_location[1], COORD_PRECISION)},{round(coordinate_location[0], COORD_PRECISION)}'
    entity_rows.append(park_features)
        
logging.info(f'Converted {len(entity_rows)} entries')
                                                        
                                                                                                        

In [ ]:
"""
Visualise the data you just generated

Hover over points to see the park name. This can be handy
jsut for verifying that everything looks roughly the way
you expect it to.
"""
from ipyleaflet import Map, Marker
from ipywidgets import HTML
import json

coords = [p['coordinate_location'].split(',') for p in park_rows]
avg_lat = sum([float(c[0]) for c in coords]) / len(coords)
avg_lon = sum([float(c[1]) for c in coords]) / len(coords)

m = Map(center=(avg_lat, avg_lon), zoom=10)

for p in entity_rows:
    coord = p['coordinate_location'].split(',')
    marker = Marker(location=(float(coord[0]), float(coord[1])), draggable=False)
    marker.title = p['ParkName']
    m.add(marker)

m

In [ ]:
"""
Build a database of potentially interesting OSM objects

NB: Currently not working for relations. There is a bit more 
processing needed to compute all the polygons of a relation.
"""

import overpass

osm_api = overpass.API()

# map of OSM tags and allowed values
# if a way or relation matches of any of these, it will be captured
target_tags = {
    'leisure': [
        'nature_reserve',
        'park',
    ],
    'landuse': [
        'forest',
        'cemetery',
    ],
    'boundary': [
        'protected_area',
    ]
}

coords = [p['coordinate_location'].split(',') for p in park_rows]
sw_lat = min([float(c[0]) for c in coords])
sw_lon = min([float(c[1]) for c in coords])

ne_lat = max([float(c[0]) for c in coords])
ne_lon = max([float(c[1]) for c in coords])

bbox = f'({sw_lat - 0.0001},{sw_lon - 0.0001},{ne_lat + 0.0001},{ne_lon + 0.0001})'

filter_strings = [f'{t}~\'{"|".join(v)}\'' for t,v in target_tags.items()]
f_parts = [f'way[{f}]{bbox}' for f in filter_strings]
f_parts += [f'relation[{f}]{bbox}' for f in filter_strings]
filter_str = ';\n'.join(f_parts) + ';'

avg_lat = sum([float(c[0]) for c in coords]) / len(coords)
avg_lon = sum([float(c[1]) for c in coords]) / len(coords)

overpass_query = f'''
        (
        {filter_str}
        );
        (._;>;);'''

logging.debug('Issuing Overpass query: %s', overpass_query)
overpass_response = osm_api.Get(overpass_query, responseformat='json')
logging.info('Downloaded all data from Overpass, beginning data cleaning')

raw_way_data = list(filter(lambda e: e['type'] != 'node', overpass_response['elements']))

# create a dict for faster lookup
node_data = {n['id']: n for n in overpass_response['elements'] if n['type'] == 'node'}

ways_data: List = list()
    
for way in raw_way_data:
    nodes = way.get('nodes', None)
    if not nodes or len(nodes) < 3:
        continue
    way_nodes = []
    for n in nodes:
        node = node_data[n]
        way_nodes.append((node['lon'], node['lat']))
    osm_object = {
        'id': way['id'],
        'type': way['type'],
        'name': way.get('tags', {}).get('name'),
        'qid': way.get('tags', {}).get('wikidata'),
        'polygon': Polygon(way_nodes),
    }
    ways_data.append(osm_object)
del raw_way_data  # bit of a memory hog
logging.info('Created a database of potential OpenStreetMap matches')



In [ ]:
"""
Reconcile between the GIS data and OSM data geospatially

Finds OSM objects that contain our chosen polylabel point.
"""
logging.info('Matching GIS objects to OSM objects')
matches = 0
for p in entity_rows:
    # there has to be a faster algo for this
    # maybe some binary search magic
    # but for now my CPU will suffer
    lat, lon = p['coordinate_location'].split(',')
    candidates = list(filter(lambda w: w['polygon'].contains(Point(lon, lat)), ways_data))
    if not candidates:
        continue
    if len(candidates) > 1:
        logging.debug(f'Found multiple potential OSM matches')
    candidate = candidates[0]
    p['osm_way_id' if candidate['type'] == 'way' else 'osm_rel_id'] = str(candidate['id'])
    p['osm_qid'] = candidate['qid']
    p['osm_name'] = candidate['name']
    matches += 1
logging.info('Finished matching GIS objects to OSM objects')
logging.info(f'Found {matches} potential matches for {len(park_rows)} records ({round((matches/len(park_rows))*100,1)}%)')


In [ ]:
"""
View and export data table
"""
import pandas

df = pandas.DataFrame.from_dict(entity_rows)
df.to_csv('target/output.csv')
df